#Python Code implementation for the Chat boat Assistant

Data for training Model is Available here.  https://drive.google.com/file/d/1apck6tkspu-Ru0Q1iNtKXaY56VQJ-jTc/view?usp=sharing

In [ ]:
pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np

# things we need for Tensorflow
import numpy
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import json
# with open('/content/drive/MyDrive/Hloov assignment/intents.json') as json_data:
#     intents = json.load(json_data)

In [ ]:
# import json
# with open('/content/drive/MyDrive/Hloov assignment/intents.json') as file:
#     data = json.load(file)
# print(data)    


{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'hours', 'patterns': ['What hours are you open?', 'What are your hours?', 'When are you open?'], 'responses': ["We're open every day 9am-9pm", 'Our hours are 9am-9pm every day']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and AMEX', 'We accept most major credit cards']}, {'tag': 'opentoday', 'patterns': ['Are you open today?', 'When do you op

In [ ]:
import json
with open('/content/drive/MyDrive/python bootcamp project/intents.json') as file:
    data = json.load(file)
print(data) 

{'intents': [{'tag': 'Identity', 'patterns': ['Who are you?', 'what are you?', 'What is your name?', 'What could I call you?', 'What can I call you?', 'What do your friends call you?', 'Tell me your name?'], 'responses': ['I am Leaf, a Discord chatbot', 'You can call me Leaf', 'You may call me Leaf', 'Call me Leaf']}, {'tag': 'greeting', 'patterns': ['Hi there', 'Is anyone there?', 'Hey', 'Good day', 'Namaste', 'yo', 'hi', 'Hi', 'Hola', 'Hello', 'Hope you are doing well?', 'Whats up', 'wassup', 'wasup', 'Wazzup', 'How are you', 'sup', 'How you doing'], 'responses': ['Hello', 'Good to see you again', 'Hi there, how can I help?', 'Hi ', 'Hola', 'Hello, I am great, how are you? ', 'Hello, how are you? I am great thanks! ', 'Hello, I am good thank you, how are you? ', 'Hi, I am great, how are you? ', 'Hi, how are you? I am great thanks! ', 'Hi, I am good thank you, how are you? ', 'Hi, good thank you, how are you? ', 'All good..What about you?'], 'context': ['']}, {'tag': 'CurrentHumanQuer

In [ ]:
words = []
labels = []
docs_x = []
docs_y = []

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [ ]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [ ]:
training = numpy.array(training)
output = numpy.array(output)

In [ ]:
# print(training)
# print(output)

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 17999  | total loss: 0.31830 | time: 0.081s
| Adam | epoch: 1000 | loss: 0.31830 - acc: 0.9861 -- iter: 136/141
Training Step: 18000  | total loss: 0.28697 | time: 0.088s
| Adam | epoch: 1000 | loss: 0.28697 - acc: 0.9875 -- iter: 141/141
--


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hi
Hi there, how can I help?
You: what do you do?
I am a general purpose chatbot. I can chat with you. Try asking me for jokes or riddles!
You: tell me a joke
What did the traffic light say to the car? "Don't look now, I'm changing."
You: make me happy
I saw this bloke chatting up a cheetah; I thought, 'He's trying to pull a fast one'.  
You: how people call you
Well I wouldn't be a clever AI bot if I did not would I?
You: how old you are?
I was made in 2022, if that's what you are asking!
You: quit
